In [1]:
import sys; sys.path.append("../")
import re
import datetime
import io

import pandas as pd

from clean.baton_rouge_fpcsb_lprr import clean
from lib.rows import duplicate_row
from lib.explore import show_unique, print_df
from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)

In [2]:
df = clean()

df

standardize_from_lookup_table: unmatched sequences:
  {'', ' - ', ' & ', '-', ' / ', '/', ' '}
../clean/baton_rouge_fpcsb_lprr.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "docket_no"] = df.docket_no.fillna("").str.strip()\
../clean/baton_rouge_fpcsb_lprr.py:103: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "hearing_date"] = df.hearing_date.fillna("").str.strip()\
../clean/baton_rouge_fpcsb_lprr.py:176: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "hearing_date"] = df.hearing_date.str.replace(r"\/\/", "/")\
../clean/baton_rouge_fpcsb_lprr.py:194: FutureWarning: The default value of regex will change from True to False in a future version.
  names = df.name.str.strip().str.replace("Tat Chi-Lam", "Tat-Chi Lam")\
../clean/baton_rouge_fpcsb_lprr.py:199: FutureWarning: The default value of regex will cha

,docket_no,action,resolution,hearing_year,hearing_month,hearing_day,counsel,rank_desc,last_name,first_name,middle_name,middle_initial,uid
0,10-03,20-day suspension,Overturned by Board (01-19-2012),2012,1,15,,,nacoste,otis,,,c0172a50191cccaf374a7391cbea298e
1,11-10,termination,"Board approved Continuance to May 17, 2012 per...",2012,2,16,,,moore,alicia,,m,69b8369b4b4750733b2785bbf54567de
2,11-11,15-day suspension,Rescheduled,2012,3,15,,,schilling,robert,,,8df55a0a8dc3fc3271efc68c0bbc60af
3,12-03,termination,Board upheld Chief White's decision to terminate,2012,4,19,,,hollis,pamela,,b,8de3a1a8d7d7169b33753fce208443ac
4,11-10,termination,Appealed to 19th Judicial Court,2012,5,17,,,moore,alicia,,m,69b8369b4b4750733b2785bbf54567de
...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,07-05,1-day suspension,Board upheld,2007,10,18,dirks,,landry iii,carroll,,j,5181a5c2da43a8f9537da12efbdf1c31
417,07-06,termination,Stayed Criminal INVestigation,2007,12,16,dirks,,davis,nathan,,,3a346b57aec3ad6d2e6d05c481ca4564
418,07-02,termination,Board Voted 4 - 1 to uphold Chief Smith,2007,12,15,tate,,handy,todd,,,32f252a6c08bdcb641b2693add02fd1b
419,07-06,termination,NaN,2007,12,20,,,davis,nathan,,,3a346b57aec3ad6d2e6d05c481ca4564


In [11]:
df = df.drop_duplicates()
df[df.duplicated(subset=['docket_no', 'uid', 'hearing_year', 'hearing_month', 'hearing_day', 'action', 'counsel'])]

,docket_no,action,resolution,hearing_year,hearing_month,hearing_day,counsel,rank_desc,last_name,first_name,middle_name,middle_initial,uid
131,01-23,1-day suspension,Did Not show - Appeal dismissed,2003,1,16,,,jarreau,tony,,,683b591126b2da2d18530c62ed5a15aa
157,00-37,letter of reprimand,Continuance granted,2001,12,20,,,johnson,lyle,,,8d77e23c58addd7d69844a4a3e7f9783
249,11-05,21-day suspension; 69-day suspension,Reduced by Board to 30 days 12-15-11 / Reduced...,2011,12,15,dirks,lieutenant,odom,caroline,,,b11bae94f7141d0b842197729bac168b
254,99-22,3-day suspension,Agreement Reached,1999,7,9,,,dehon,carl,,,9975a2784c6461789ef25079e0f45922
259,99-27,letter of reprimand,NaN,,,,,,hammond,scott,,,8ba08339b9ff73e2a7878c04e520333f
342,09-10,rehearing of appeal,No reason to appeal,,,,,,rucker,tumaini,,,f05511a714c04234a40fec1b29c0e65a
344,10-01,termination,10/28 Lotwick Cont'd,2010,10,28,dirks,officer,moruzzi,robert,,,40d911f2198a0fb16fd296d82a46a9f0
348,09-01,termination,NaN,2010,12,16,,,webb,jeffrey,,,7bc062a58cc1f366a32fdc1a531baeca
363,07-06,termination,Pending Criminal Investigation,,,,dirks iii,,davis,nathan,,,3a346b57aec3ad6d2e6d05c481ca4564
378,07-03,3-day suspension,NaN,2009,4,2,dirks iii,,alexander,carl,,,67a3ab1dd89c539cdcb914de8d6438d6


In [10]:
df[df.docket_no == '09-11']

,docket_no,action,resolution,hearing_year,hearing_month,hearing_day,counsel,rank_desc,last_name,first_name,middle_name,middle_initial,uid
235,09-11,transfer,Dirks withdrew,,,,dirks,,welsh iii,clarence,,k,499dd94caa85e5fb6d2b9e02546b0f84
343,09-11,transfer,Dirks withdrew,,,,,,welsh iii,clarence,,k,499dd94caa85e5fb6d2b9e02546b0f84


In [27]:
pprr17 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2017.csv')

dfa = df[['uid', 'first_name', 'last_name', 'middle_initial']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = pprr17[['uid', 'first_name', 'last_name', 'middle_initial']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
    "middle_initial": StringSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=5, step=0.025))

                                                                    first_name     last_name middle_initial fc
score_range pair_idx sim_score row_key                                                                        
1.00-0.97   0        1.000000  69b8369b4b4750733b2785bbf54567de         alicia         moore              m  a
                               5643bfcab8a70cd0a787c1d89faa37c4         alicia         moore              m  a
            1        1.000000  6bd14ff44edb7ea8b7c2b585abf0655d         darryl        honore              w  d
                               4004d4ff0d896b440bfad9a32923383a         darryl        honore              w  d
            2        1.000000  bdf60120a8cc6acbe1a5bac9270b23e0         donald       shirley              r  d
                               dbe7b6574b3d38399969573690a9f57e         donald       shirley              r  d
            3        1.000000  142453c6e73c1da8c86ba08a365404e8         samuel         white              d  s
 

In [28]:
pprr19 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2019.csv')

dfa = df[['uid', 'first_name', 'last_name', 'middle_initial']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = pprr19[['uid', 'first_name', 'last_name', 'middle_initial']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
    "middle_initial": StringSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=5, step=0.025))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
                                                                first_name   last_name middle_initial fc
score_range pair_idx sim_score row_key                              

In [30]:
post = pd.read_csv('../data/clean/pprr_post_2020_11_06.csv')
post = post[post.agency == 'baton rouge pd']

dfa = df[['uid', 'first_name', 'last_name']]
dfa.loc[:, 'fc'] = dfa.first_name.map(lambda x: x[:1])
dfa = dfa.drop_duplicates(subset=['uid']).set_index('uid')

dfb = post[['uid', 'first_name', 'last_name']]
dfb.loc[:, 'fc'] = dfb.first_name.map(lambda x: x[:1])
dfb = dfb.drop_duplicates(subset=['uid']).set_index('uid')

matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(["fc"]), {
    "first_name": JaroWinklerSimilarity(),
    "last_name": JaroWinklerSimilarity(),
})
print_df(matcher.get_sample_pairs(sample_counts=10, step=0.025))

/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/khoipham/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
                                                                  first_name     last_name fc
score_range pair_idx sim_score row_key                                         

In [32]:
len(matcher.get_index_pairs_within_thresholds(0.93))

86